#CMPE 297 - Assignment 5 - Part 3
##Abhishek Bais
**Acknowledgements**  
[colab](https://github.com/ContinualAI/avalanche/blob/master/notebooks/from-zero-to-hero-tutorial/07_putting-all-together.ipynb)  
**Dataset:** omniglot

#1.0 Install avalanche

In [1]:
!pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-d45a8ofq
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-d45a8ofq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 121 kB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 532 kB 22.6 MB/s 
     |████████████████████████████████| 1.7 MB 28.5 MB/s 
     |████████████████████████████████| 180 kB 50.6 MB/s 
     |████████████████████████████████| 97 kB 6.5 MB/s 
     |████████████████████████████████| 139 kB 50.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for avalanche-lib: filename=avalanche_lib-0.0.1-py3-none-any.whl size=414217 sha256=818c94c6bc96b9fa9904747cea5e27a5d99f5a186e2168736fab3560e

#2.0 Load libaries

In [22]:
import torch
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.datasets.omniglot import Omniglot
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive
from torchvision import transforms
from avalanche.benchmarks import nc_benchmark

# 3.0 Load dataset

In [5]:
dataset_root = "omniglot"
omniglot_train = Omniglot(root=dataset_root, train=True, download=True)
omniglot_test = Omniglot(root=dataset_root, train=False, download=True)

  0%|          | 0/9464212 [00:00<?, ?it/s]

Extracting omniglot/omniglot-py/omniglot-py/images_background.zip to omniglot/omniglot-py/omniglot-py


  0%|          | 0/6462886 [00:00<?, ?it/s]

Extracting omniglot/omniglot-py/omniglot-py/images_evaluation.zip to omniglot/omniglot-py/omniglot-py


# 4.0 Define the hyper parameters

In [31]:
n_classes = 964
size = 32
n_experiences = 4 # need a number that dividies 964
seed = 101

# 5.0 Build the model

In [28]:
# MODEL CREATION
model = SimpleMLP(num_classes=n_classes, input_size=size ** 2)

transform = transforms.Compose([
    transforms.Resize(size),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.9221,), (0.2681,))
])

scenario = nc_benchmark(
    train_dataset=omniglot_train,
    test_dataset=omniglot_test,
    n_experiences=n_experiences,
    task_labels=False,
    seed=seed,
    fixed_class_order=None,
    shuffle=True,
    train_transform=transform,
    eval_transform=transform
)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:84: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "


# 6.0 Train the model

In [30]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=8, train_epochs=5, eval_mb_size=8,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience, num_workers=4)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [3, 515, 5, 6, 517, 8, 518, 13, 14, 525, 16, 526, 527, 530, 534, 23, 535, 536, 538, 539, 28, 540, 30, 541, 542, 544, 546, 35, 549, 38, 39, 551, 556, 45, 48, 51, 564, 566, 567, 58, 578, 580, 581, 72, 73, 584, 587, 76, 81, 602, 91, 603, 607, 608, 100, 614, 619, 109, 623, 624, 115, 628, 629, 631, 120, 121, 632, 633, 640, 130, 642, 132, 647, 649, 138, 650, 140, 657, 661, 151, 152, 153, 663, 667, 159, 161, 673, 677, 166, 167, 681, 171, 685, 175, 689, 179, 181, 192, 195, 708, 198, 711, 712, 201, 204, 207, 720, 210, 727, 220, 221, 734, 226, 740, 747, 238, 750, 243, 244, 755, 758, 247, 250, 251, 763, 253, 765, 257, 258, 770, 261, 265, 778, 791, 792, 794, 291, 292, 296, 809, 299, 812, 303, 304, 305, 816, 307, 308, 311, 823, 313, 316, 828, 831, 832, 834, 838, 328, 840, 330, 842, 844, 333, 850, 339, 851, 856, 859, 860, 861, 865, 868, 363, 364, 878, 884, 373, 374, 375, 889, 378, 379, 898, 388, 390, 902, 393, 905, 907, 912, 913, 402, 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 603/603 [07:52<00:00,  1.28it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 79270.9824
	DiskUsage_MB/train_phase/train_stream/Task000 = 79270.9824
	Loss_Epoch/train_phase/train_stream/Task000 = 3.7268
	Loss_MB/train_phase/train_stream/Task000 = 3.7889
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0003
	Time_Epoch/train_phase/train_stream/Task000 = 471.3682
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2249
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.2500
100%|██████████| 603/603 [07:48<00:00,  1.29it/s]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 79469.9512
	DiskUsage_MB/train_phase/train_stream/Task000 = 79469.9512
	Loss_Epoch/train_phase/train_stream/Task000 = 2.9826
	Loss_MB/train_phase/train_stream/Task000 = 2.0665
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0003
	Time_Epoch/train_phase/train_stream/Task000 = 467.6961
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3446
	Top1_Acc